In [ ]:
import cx_Oracle
from os import getenv

import numpy as np
import pandas as pd
from pandas.tseries.offsets import Day

import datetime, json
import os
from google.cloud import secretmanager


import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from matplotlib.pyplot import figure
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (20,12)

import plotly 
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import iplot
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def set_secrets_as_envs():
  secrets = secretmanager.SecretManagerServiceClient()
  resource_name = f"{os.environ['KNADA_TEAM_SECRET']}/versions/latest"
  secret = secrets.access_secret_version(name=resource_name)
  secret_str = secret.payload.data.decode('UTF-8')
  secrets = json.loads(secret_str)
  os.environ.update(secrets)

In [ ]:
def oracle_secrets():
  set_secrets_as_envs()
  return dict(
    user=os.getenv("UID"),
    password=os.getenv("PWD"),
    host = os.getenv("HOST"),
    service = os.getenv("SERVICE"),
    encoding="UTF-8",
    nencoding="UTF-8"
  )

oracle_secrets = oracle_secrets()

In [ ]:
def get_data(sql):
    user = oracle_secrets['user'] + '[DVH_FAM_PP]' #f"[{schema}]"
    dsn_tns = cx_Oracle.makedsn(oracle_secrets['host'], 1521, service_name = oracle_secrets['service'])
    try:
        conn = cx_Oracle.connect(user = user, password = oracle_secrets['password'], dsn = dsn_tns)
        df = pd.read_sql(sql = sql, con = conn)
        conn.close()
        return df
    except cx_Oracle.Error as error:
        print(error)

In [ ]:
sql = """ 
SELECT A.PERIODE, DIM_TID.DATO, A.UTBET_FOM, A.UTBET_TOM, A.FK_PERSON1_MOTTAKER, A.FK_PERSON1_PLEIETRENGENDE, A.UTFALL
,A.SAKSNUMMER, A.ARBEIDSFORHOLD_TYPE, A.PLEIEBEHOV
,B.GMT_OVERSE_ETABLERT_TILSYN_AARSAK, B.SOKERS_TAPTE_ARBEIDSTID, B.GMT_ANDRE_SOKERS_TILSYN, B.GMT_TILGJENGELIG_FOR_SOKER, B.GMT_ETABLERT_TILSYN, B.NATTEVAAK, B.BEREDSKAP
,AARSAK.AARSAK
FROM fam_pp_stonad A
JOIN fam_pp_perioder B ON A.FK_PP_PERIODER=B.PK_PP_PERIODER
LEFT OUTER JOIN fam_pp_periode_aarsak AARSAK ON AARSAK.FK_PP_PERIODER=B.PK_PP_PERIODER 
JOIN DT_P.DIM_TID DIM_TID ON DIM_TID.DIM_NIVAA=1 
AND DIM_TID.DAG_I_UKE<6 
AND DIM_TID.GYLDIG_FLAGG=1 
AND A.UTBET_FOM<=DIM_TID.DATO 
AND A.UTBET_TOM>=DIM_TID.DATO
WHERE A.KILDESYSTEM='PP_VEDTAK' AND A.GYLDIG_FLAGG=0 AND A.PERIODE>202200 AND A.PERIODE<202205
and to_char(DIM_TID.DATO,'YYYYMM')=PERIODE"""

df = get_data(sql = sql)

In [ ]:
df.head()

# Bestilling av stønadsstatistikk i forbindelse med endringer i 9-11

- Trenger totalt antall saker per dag (som er i søknadsperioden).
- Hvilken tidsperiode uttrekket gjelder for
    - Fullt uttrekk for alle data fra K9 
    - Data per dag dataene gjelder for.

### Total antall saker per dag innvilget og ikke innvilget 

In [ ]:
antal_saker = df[['DATO', 'SAKSNUMMER', 'UTFALL']].pivot_table(index = 'DATO', columns = 'UTFALL',values = 'SAKSNUMMER', aggfunc = lambda x: len(x.unique()), margins = True)[:-1]

In [ ]:
# gir navn til kolonnene

antal_saker.columns = [''] * len(antal_saker.columns)
antal_saker.columns = ['ANT_SAKER_IKKE_OPP', 'ANT_SAKER_OPP', 'ANT_SAK_TOTALT'] 
antal_saker.head()

- I hvor mange saker er det opplyst om timer med tilsynsordning?
    - Antall saker med tilsynsordning per dag.

### Antall saker med tilsynsordning per dag

In [ ]:
# saker som har utfall som opppfylt og har tilsyn (GMT_ETABLERT_TILSYN > 0)

filt = ((df['UTFALL'] == 'OPPFYLT') & (df['GMT_ETABLERT_TILSYN'] > 0)) 
tilsynsordning = df.loc[filt][['DATO', 'SAKSNUMMER']].pivot_table(
    index = 'DATO', values = 'SAKSNUMMER' ,aggfunc = lambda x: len(x.unique()))

In [ ]:
tilsynsordning.columns = [''] * len(tilsynsordning.columns)
tilsynsordning.columns =['Ant_SAK_MED_TILSYN']
tilsynsordning.head()

- I hvor mange av sakene med tilsynsordning er det vurdert nattevåk og/eller beredskap?
    - Antall saker med innvilget beredskap per dag.
    - Antall saker med avslått beredskap per dag.
    - Antall saker med innvilget nattevåk per dag.
    - Antall saker med avslått nattevåk per dag.
    - Antall saker med innvilget beredskap & nattevåk per dag.
    - Antall saker med avslått beredskap & nattevåk per dag.

In [ ]:
filt = ((df['UTFALL'] == 'OPPFYLT') & (df['GMT_ETABLERT_TILSYN'] > 0))
beredskap_df = df.loc[filt, ['DATO','SAKSNUMMER', 'BEREDSKAP']].pivot_table(index = 'DATO', columns = 'BEREDSKAP',values = 'SAKSNUMMER', aggfunc = lambda x: len(x.unique()))
beredskap_df.columns = [''] * len(beredskap_df.columns)
beredskap_df.columns = ['BEREDSKAP_IKKE_OPP', 'BEREDSKAP_OPP']
beredskap_df.head()

In [ ]:
filt = ((df['UTFALL'] == 'OPPFYLT') & (df['GMT_ETABLERT_TILSYN'] > 0))
nattevaak_df = df.loc[filt, ['DATO','SAKSNUMMER', 'NATTEVAAK']].pivot_table(index = 'DATO', columns = 'NATTEVAAK',values = 'SAKSNUMMER', aggfunc = lambda x: len(x.unique()))
nattevaak_df.columns = [''] * len(nattevaak_df.columns)
nattevaak_df.columns = ['NATTEVAAK_IKKE_OPP', 'NATTEVAAK_OPP']
nattevaak_df.head()

In [ ]:
filt = ((df['BEREDSKAP'] == 'OPPFYLT') & (df['NATTEVAAK'] == 'OPPFYLT') & (df['UTFALL'] == 'OPPFYLT') & (df['GMT_ETABLERT_TILSYN'] > 0))
nattevaak_og_beredskap_opp_df = df.loc[filt][['DATO', 'SAKSNUMMER']].pivot_table(index = 'DATO', values = 'SAKSNUMMER', aggfunc = lambda x: len(x.unique()))

In [ ]:
nattevaak_og_beredskap_opp_df.columns = [''] * len(nattevaak_og_beredskap_opp_df.columns)
nattevaak_og_beredskap_opp_df.columns = ['BEREDSKAP_OG_NATTEVAAK_INNVILGET']
nattevaak_og_beredskap_opp_df.head()

In [ ]:
filt = ((df['BEREDSKAP'] == 'IKKE_OPPFYLT') & (df['NATTEVAAK'] == 'IKKE_OPPFYLT') & (df['UTFALL'] == 'OPPFYLT') & (df['GMT_ETABLERT_TILSYN'] > 0))
nattevaak_og_beredskap_ikke_opp_df = df.loc[filt][['DATO', 'SAKSNUMMER']].pivot_table(index = 'DATO', values = 'SAKSNUMMER', aggfunc = lambda x: len(x.unique()))

In [ ]:
# denne spørringen kun viser rader som har sak (så hvis en rad har ikke noe sak så vises ikke raden her).

nattevaak_og_beredskap_ikke_opp_df.columns = [''] * len(nattevaak_og_beredskap_opp_df.columns)
nattevaak_og_beredskap_ikke_opp_df.columns = ['BEREDSKAP_OG_NATTEVAAK_IKKE_INNVILGET']
nattevaak_og_beredskap_ikke_opp_df

In [ ]:
# kombinerte de 6 koloner 

natt_bred_result = pd.concat([beredskap_df, nattevaak_df, nattevaak_og_beredskap_opp_df, nattevaak_og_beredskap_ikke_opp_df], 
                             axis=1).reindex(beredskap_df.index).fillna(value = 0)
natt_bred_result.astype(int).head()

- I hvor mange av sakene er det unntak fra gradering på grunn av nattevåk og/eller beredskap?
    - Antall saker der nattevåk og/eller beredskap blir brukt som unntak per dag.

In [ ]:
# fortsatt litt usikker på om (df['GMT_ETABLERT_TILSYN'] > 0) skal være med eller ikke?

l = ['NATTEVÅK_OG_BEREDSKAP', 'BEREDSKAP', 'NATTEVÅK']
#filt = ((df['GMT_ETABLERT_TILSYN'] > 0) & (df['UTFALL'] == 'OPPFYLT') & (df['GMT_OVERSE_ETABLERT_TILSYN_AARSAK'].isin(l)))
filt = ( (df['UTFALL'] == 'OPPFYLT') & (((df['NATTEVAAK'] == 'OPPFYLT') | (df['BEREDSKAP'] == 'OPPFYLT')) | ((df['NATTEVAAK'] == 'OPPFYLT') & (df['BEREDSKAP'] == 'OPPFYLT')))) 
natt_bred_unntak = df.loc[filt][['DATO', 'SAKSNUMMER']].pivot_table(index='DATO', values = 'SAKSNUMMER', aggfunc = lambda x: len(x.unique()) )

In [ ]:
natt_bred_unntak.columns = [''] * len(natt_bred_unntak.columns)
natt_bred_unntak.columns = ['ANT_SAK_NATT_OG_PRED_UNNTAK_FRA_GRAD']
natt_bred_unntak.head()

- Antall saker som graderes med årsak per dag:
    - GRADERT_MOT_TILSYN
    - AVKORTET_MOT_INNTEKT

In [ ]:
# utfall oppfylt, aarsak er enten AVKORTET_MOT_INNTEKT eller GRADERT_MOT_TILSYN

filt = ((df['UTFALL'] == 'OPPFYLT') & ((df['AARSAK'] == 'AVKORTET_MOT_INNTEKT') | (df['AARSAK'] == 'GRADERT_MOT_TILSYN')))
gradering_med_aarsak = df.loc[filt, ['DATO', 'AARSAK', 'SAKSNUMMER']].pivot_table(index = 'DATO', columns = 'AARSAK', values = 'AARSAK',
                                                           aggfunc = lambda x: len(x.unique()), fill_value=0)

In [ ]:
gradering_med_aarsak.columns = [''] * len(gradering_med_aarsak.columns)
gradering_med_aarsak.columns =['ANT_SAK_AVKORTET_MOT_INNTEKT', 'ANT_SAK_GRADERT_MOT_TILSYN']
gradering_med_aarsak.head()

- Antall avslag med årsak per dag:
    - FOR_LAV_REST_PGA_ETABLERT_TILSYN 
    - FOR_LAV_REST_PGA_ETABLERT_TILSYN_OG_ANDRE_SØKERE
    - FOR_LAV_TAPT_ARBEIDSTID
    - FOR_LAV_REST_PGA_ANDRE_SØKERE

In [ ]:
# om du legger til UTFALL = 'OPPFYLT' så får du samme svar så den er ikke nødvendig her 

avslag_aarsak_list = ['FOR_LAV_REST_PGA_ETABLERT_TILSYN', 'FOR_LAV_REST_PGA_ETABLERT_TILSYN_OG_ANDRE_SØKERE', 'FOR_LAV_TAPT_ARBEIDSTID', 'FOR_LAV_REST_PGA_ANDRE_SØKERE']
filt = df['AARSAK'].isin(avslag_aarsak_list)
avslag_aarsak = df.loc[filt, ['DATO', 'AARSAK', 'SAKSNUMMER']].pivot_table(index = 'DATO', columns = 'AARSAK', values = 'SAKSNUMMER',
                                                          aggfunc = lambda x: len(x.unique()), fill_value = 0)

In [ ]:
avslag_aarsak.columns = [''] * len(avslag_aarsak.columns)
avslag_aarsak.columns =['ANT_SAK_FOR_LAV_REST_PGA_ANDRE_SØKERE', 'ANT_SAK_FOR_LAV_REST_PGA_ETABLERT_TILSYN', 'ANT_SAK_FOR_LAV_REST_PGA_ETABLERT_TILSYN_OG_ANDRE_SØKERE', 'ANT_SAK_FOR_LAV_TAPT_ARBEIDSTID']
avslag_aarsak.head()

- Hvor mange har ikke arbeid?
    - Alle aktiviteter bortsett fra selvstendig næringsdrivende, frilanser og/eller arbeidstaker.

In [ ]:
# saker som ikk her en av de kodene 'AT','SN','FL', None (ikke ta med null rader heller)

bortsett_liste = ['AT','SN','FL', None]
filt = (df['UTFALL'] == 'OPPFYLT') & (~df['ARBEIDSFORHOLD_TYPE'].isin(bortsett_liste))
ikke_arbeid = df.loc[filt, ['DATO', 'SAKSNUMMER']].pivot_table(index = 'DATO',values = 'SAKSNUMMER', aggfunc = lambda x: len(x.unique())
                                                ,fill_value = 0)

In [ ]:
ikke_arbeid.columns = [''] * len(ikke_arbeid.columns)
ikke_arbeid.columns =['ANT_SAK_UTEN_ARBEID']
ikke_arbeid

- Hvor mange har ikke arbeid og oppgir tilsyn?
    - Antall saker med arbeid og oppgitt tilsyn per dag.

In [ ]:
arbeid_liste = ['AT','SN','FL']
filt = ((df['GMT_ETABLERT_TILSYN'] > 0) & (df['UTFALL'] == 'OPPFYLT') & (~df['ARBEIDSFORHOLD_TYPE'].isin(arbeid_liste)))
saker_arbeid_tilsyn = df.loc[filt, ['DATO', 'SAKSNUMMER']].pivot_table(index = 'DATO', values = 'SAKSNUMMER',
                                                aggfunc = lambda x: len(x.unique()), fill_value = 0)

In [ ]:
saker_arbeid_tilsyn.columns = [''] * len(saker_arbeid_tilsyn.columns)
saker_arbeid_tilsyn.columns =['ANT_SAK_IKKE_ARBEID_OG_OPPGIR_TILSYN']
saker_arbeid_tilsyn

## Gradering mot annen omsorgsperson:

- Hvor mange saker har flere søkere i samme periode?
    - Antall saker for et gitt antall søkere (1,2,3,4 eller 5) per dag. (ikke så viktig)

- I hvor mange saker er det kun behov for 1 omsorgsperson?
    - Antall saker for en gitt pleiegrad (0%, 100%, eller 200%) per dag.

In [ ]:
# hvis vi vil ha de som har 0% så må vi ikke begrense til UTFALL = 'OPPFYLLT'

filt = (df['UTFALL'] == 'OPPFYLT')
pleiebehov = df.loc[filt,['DATO', 'PLEIEBEHOV', 'SAKSNUMMER']].pivot_table(index = 'DATO',columns = 'PLEIEBEHOV', values = 'SAKSNUMMER',
                                                    aggfunc = lambda x: len(x.unique()), fill_value=0)

In [ ]:
pleiebehov.columns = [''] * len(pleiebehov.columns)
pleiebehov.columns =['ANT_SAK_100_PLEIEBEHOV', 'ANT_SAK_200_PLEIEBEHOV']
pleiebehov

## Gradering mot arbeid:

- Dette uttrekket er ikke per dag, men totalt over alle periodene til en fagsak:
    - Hvor mange saker graderes/avslås uten årsakene AVKORTET_MOT_INNTEKT og/eller FOR_LAV_TAPT_ARBEIDSTID
    - Hvor mange saker graderes/avslås uten årsaken GRADERT_MOT_TILSYN

### Antall saker graderes/avslås uten årsakene AVKORTET_MOT_INNTEKT og/eller FOR_LAV_TAPT_ARBEIDSTID

In [ ]:
# ifølge Stian så skal vi ikke bruke noe annet kolonne enn AARSAK

bortsett_aarsak = ['AVKORTET_MOT_INNTEKT', 'FOR_LAV_TAPT_ARBEIDSTID']
filt = (~df['AARSAK'].isin(bortsett_aarsak)) 

df.loc[filt][['UTFALL', 'SAKSNUMMER']].pivot_table(
    index = 'UTFALL', values = 'SAKSNUMMER' ,aggfunc = lambda x: len(x.unique()), fill_value=0)

### Antall saker graderes/avslås uten årsaken GRADERT_MOT_TILSYN

In [ ]:
filt = (df['AARSAK'] != 'GRADERT_MOT_TILSYN') 
df.loc[filt,['UTFALL', 'SAKSNUMMER']].pivot_table(index = 'UTFALL', values = 'SAKSNUMMER', aggfunc = lambda x: len(x.unique()))

## §8-13 og gradering mot 20 %

 - Antall saker med årsak FOR_LAV_TAPT_ARBEIDSTID per dag

In [ ]:
filt = (df['AARSAK'] == 'FOR_LAV_TAPT_ARBEIDSTID')
df.loc[filt, ['DATO', 'SAKSNUMMER']].pivot_table(index = 'DATO', values = 'SAKSNUMMER', aggfunc = lambda x: len(x.unique()), fill_value = 0)

 - Antallet saker som ikke har minst én søknadsperiode som er lenger enn 5 dager sammenhengende.

### Antallet saker som ikke har minst én søknadsperiode som er lenger enn 5 dager sammenhengende.

In [ ]:
sql_ = """ 
SELECT A.PERIODE, A.UTBET_FOM, A.UTBET_TOM, A.FK_PERSON1_MOTTAKER, A.FK_PERSON1_PLEIETRENGENDE, A.UTFALL, A.OPPGITT_TILSYN
,A.SAKSNUMMER, A.ARBEIDSFORHOLD_TYPE, A.PLEIEBEHOV
,B.GMT_OVERSE_ETABLERT_TILSYN_AARSAK, B.GMT_ETABLERT_TILSYN
,AARSAK.AARSAK
FROM fam_pp_stonad A
JOIN fam_pp_perioder B ON A.FK_PP_PERIODER=B.PK_PP_PERIODER
LEFT OUTER JOIN fam_pp_periode_aarsak AARSAK ON AARSAK.FK_PP_PERIODER=B.PK_PP_PERIODER 
WHERE A.KILDESYSTEM='PP_VEDTAK' AND A.GYLDIG_FLAGG=0 AND A.PERIODE>202200 AND A.PERIODE<202205
"""
df_ = get_data(sql = sql_)

In [ ]:
df_ = df_[['PERIODE','UTBET_FOM', 'UTBET_TOM', 'SAKSNUMMER']]

In [ ]:
df_['ANTALL_DAGER'] = df_['UTBET_TOM'].sub(df_['UTBET_FOM']).mod(365*Day()).dt.days + 1

In [ ]:
# hent alle saker som har én søknadsperiode som er lik 5

filt = (df_['ANTALL_DAGER'] == 5)
saker_med_søknadsperiode_5 = df_.loc[filt, 'SAKSNUMMER'].unique() # liste med alle saker som har én søknadsperiode på 5 dager i alle periodene januar, februer, mars, april

In [ ]:
# henter kun saker som ikke er i saker_med_søknadsperiode_5

filt = (~(df_['SAKSNUMMER'].isin(saker_med_søknadsperiode_5)))
df_.loc[filt]

In [ ]:
df_['WEEK_UTBET_FOM'] = df_['UTBET_FOM'].dt.week
df_['WEEK_UTBET_TOM'] = df_['UTBET_TOM'].dt.week
filt = (df_['WEEK_UTBET_FOM'] != df_['WEEK_UTBET_TOM'])
#df_.loc[filt]

In [ ]:
# test case
#df_.loc[(df_['SAKSNUMMER'] == 'ABX74') & (df_['WEEK_UTBET_FOM'] == 1)]
#df_.loc[(df_['SAKSNUMMER'] == 'ABX74') & (df_['WEEK_UTBET_FOM'] == 1)].drop_duplicates(keep='first').groupby(['WEEK_UTBET_FOM', 'SAKSNUMMER'])['ANTALL_DAGER'].sum()

In [ ]:
filt = (~(df_['SAKSNUMMER'].isin(saker_med_søknadsperiode_5)))
x = df_.loc[filt].drop_duplicates(keep='first').groupby(['WEEK_UTBET_FOM', 'SAKSNUMMER'])['ANTALL_DAGER'].sum().reset_index()
x

In [ ]:
# Antall saker som ikke har minst én søknadsperiode som er lenger enn 5 dager sammenhengende
len(x[x['ANTALL_DAGER']<5])

### Antall nye saker per periode

In [ ]:
nye_saker = df[['PERIODE', 'SAKSNUMMER']].drop_duplicates(keep='first').sort_values(by = 'PERIODE')
nye_saker['PERIODE'] = pd.to_datetime(nye_saker['PERIODE'],format='%Y%m') 

In [ ]:
s1 = nye_saker.groupby('PERIODE')['SAKSNUMMER'].value_counts()
s2 = s1.unstack().diff().replace({0: False, np.nan: True}).stack()
nye_saker['NY_SAK'] = nye_saker.set_index(['PERIODE', 'SAKSNUMMER']).index.map(s2)

In [ ]:
nye_saker

In [ ]:
filt = (nye_saker['NY_SAK'] == True) & (nye_saker['PERIODE'] > '2022-01-01')
nye_saker.loc[filt]

In [ ]:
ant_ny_sak_per_periode = pd.crosstab(nye_saker.loc[filt, 'PERIODE'], nye_saker.loc[filt, 'NY_SAK'])

In [ ]:
ant_ny_sak_per_periode.columns = [''] * len(ant_ny_sak_per_periode.columns)
ant_ny_sak_per_periode.columns = ['ANT_NYE_SAKER']
ant_ny_sak_per_periode

In [ ]:
filt = (df['PERIODE'] == '202202')
len(df.loc[filt, 'SAKSNUMMER'].unique())

### Resultat tabell

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
slutt_df = pd.concat([antal_saker,tilsynsordning,natt_bred_result,natt_bred_unntak,gradering_med_aarsak,avslag_aarsak,ikke_arbeid,saker_arbeid_tilsyn,pleiebehov], 
                             axis=1).reindex(antal_saker.index).fillna(value = 0)

In [ ]:
slutt_df

In [ ]:
slutt_df.to_excel('resultat_tabell.xlsx')

In [ ]:
------------------------- Ikke se på resten her, bare data jeg leker med ---------------------

In [ ]:
df_behov = df[['PLEIEBEHOV', 'GMT_ANDRE_SOKERS_TILSYN', 'GMT_TILGJENGELIG_FOR_SOKER']].astype(float)
df_behov

In [ ]:
df_behov = df[['PLEIEBEHOV', 'GMT_ANDRE_SOKERS_TILSYN', 'GMT_TILGJENGELIG_FOR_SOKER']].astype(float)
df_behov['tilsyn_sum'] = df_behov['GMT_ANDRE_SOKERS_TILSYN'] + df['GMT_TILGJENGELIG_FOR_SOKER']
df_behov[df_behov['PLEIEBEHOV'] != df_behov['tilsyn_sum']]

In [ ]:
pivot = df[['DATO', 'SAKSNUMMER', 'FK_PERSON1_MOTTAKER']].pivot_table(index = ['DATO', 'SAKSNUMMER']
                                                                        ,values = 'FK_PERSON1_MOTTAKER', aggfunc = lambda x: len(x.unique()))

In [ ]:
pivot.reset_index(inplace = True)

In [ ]:
pivot.loc[filt].groupby(['DATO',''])['SAKSNUMMER'].nunique()

In [ ]:
filt = ((df['GMT_OVERSE_ETABLERT_TILSYN_AARSAK'] != 'FOR_LAVT') & df['GMT_OVERSE_ETABLERT_TILSYN_AARSAK'].notna()) # henter saker med tilsynsordning

tilsynsordning_med_aarsak = df.loc[filt][['DATO', 'SAKSNUMMER', 'UTFALL', 'GMT_OVERSE_ETABLERT_TILSYN_AARSAK']].pivot_table(
    index = ['DATO', 'UTFALL'], columns = 'GMT_OVERSE_ETABLERT_TILSYN_AARSAK', values = 'SAKSNUMMER' ,aggfunc = lambda x: len(x.unique()), 
                    fill_value=0).rename_axis(None, axis = 1).unstack()

In [ ]:
tilsynsordning_med_aarsak.columns = [''] * len(tilsynsordning_med_aarsak.columns)
tilsynsordning_med_aarsak.columns =['BEREDSKAP_IKKE_OPP', 'BEREDSKAP_OPP','NATTEVÅK_IKKE_OPP', 'NATTEVÅK_OPP', 'NATTEVÅK_OG_BEREDSKAP_IKKE_OPP', 'NATTEVÅK_OG_BEREDSKAP_OPP']
tilsynsordning_med_aarsak

In [ ]:
filt = ((df['UTFALL'] == 'OPPFYLT') & (df['GMT_ETABLERT_TILSYN'].notna()) & (df['GMT_ETABLERT_TILSYN'] == 0) 
        & (df['GMT_OVERSE_ETABLERT_TILSYN_AARSAK'] != 'FOR_LAVT') & (df['GMT_OVERSE_ETABLERT_TILSYN_AARSAK'].notna()))
untak_fra_gradering_nattvåk_og_beresdkap = df.loc[filt][['DATO', 'SAKSNUMMER']].pivot_table(
    index = 'DATO', values = 'SAKSNUMMER' ,aggfunc = lambda x: len(x.unique()), fill_value=0)

In [ ]:
untak_fra_gradering_nattvåk_og_beresdkap.columns = [''] * len(untak_fra_gradering_nattvåk_og_beresdkap.columns)
untak_fra_gradering_nattvåk_og_beresdkap.columns = ['ANT_SAK_NATTVAAK_BEREDSKAP_BRUKT_SOM_UNNTAK']
untak_fra_gradering_nattvåk_og_beresdkap

In [ ]:
%history